In [1]:
# importing dependencies
import pandas as pd # data frame
import numpy as np # matrix math
from scipy.io import wavfile # reading the wavfile
import os # interation with the OS
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
import matplotlib.pyplot as plt # to view graphs
import wave
from math import log, floor
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
from keras.utils import *
from sklearn.ensemble import *
import pickle

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
X_train = np.load('feature/mfcc/X_train.npy')
X_train2 = np.load('feature/fbank/X_train.npy')
print(X_train.shape,X_train2.shape)

(9473, 40, 345, 1) (9473, 128, 1034, 1)


In [5]:
SAMPLE_RATE = 44100
fname = 'data/audio_train/002d256b.wav'    # Hi-hat
wav, _ = librosa.core.load(fname, sr=SAMPLE_RATE)
# wav = wav[:2*44100]
# mfcc = librosa.feature.mfcc(wav, sr = SAMPLE_RATE, n_mfcc=400)
wav.shape
#取八秒

(19404,)

In [3]:
Y_train = pd.read_csv("data/train_label.csv")
labels = list(Y_train['label'].unique())
labels

['Hi-hat',
 'Saxophone',
 'Trumpet',
 'Glockenspiel',
 'Cello',
 'Knock',
 'Gunshot_or_gunfire',
 'Clarinet',
 'Computer_keyboard',
 'Keys_jangling',
 'Snare_drum',
 'Writing',
 'Laughter',
 'Tearing',
 'Fart',
 'Oboe',
 'Flute',
 'Cough',
 'Telephone',
 'Bark',
 'Chime',
 'Bass_drum',
 'Bus',
 'Squeak',
 'Scissors',
 'Harmonica',
 'Gong',
 'Microwave_oven',
 'Burping_or_eructation',
 'Double_bass',
 'Shatter',
 'Fireworks',
 'Tambourine',
 'Cowbell',
 'Electric_piano',
 'Meow',
 'Drawer_open_or_close',
 'Applause',
 'Acoustic_guitar',
 'Violin_or_fiddle',
 'Finger_snapping']

In [4]:
dicts={}
for i,v in enumerate(labels):
    dicts[v]=i
Y_train['trans'] = Y_train['label'].map(dicts)
Y_train

,fname,label,manually_verified,trans
0,00044347.wav,Hi-hat,0,0
1,001ca53d.wav,Saxophone,1,1
2,002d256b.wav,Trumpet,0,2
3,0033e230.wav,Glockenspiel,1,3
4,00353774.wav,Cello,1,4
5,003b91e8.wav,Cello,0,4
6,003da8e5.wav,Knock,1,5
7,0048fd00.wav,Gunshot_or_gunfire,1,6
8,004ad66f.wav,Clarinet,0,7
9,0063ab88.wav,Computer_keyboard,0,8


In [19]:
# pickle.dump(dicts,open('data/map.pkl','wb'))
# pickle.dump(dicts_ch,open('data/map_reverse.pkl','wb'))

In [21]:
# pickle.load(open('data/map_reverse.pkl','rb'))

{0: 'Hi-hat',
 1: 'Saxophone',
 2: 'Trumpet',
 3: 'Glockenspiel',
 4: 'Cello',
 5: 'Knock',
 6: 'Gunshot_or_gunfire',
 7: 'Clarinet',
 8: 'Computer_keyboard',
 9: 'Keys_jangling',
 10: 'Snare_drum',
 11: 'Writing',
 12: 'Laughter',
 13: 'Tearing',
 14: 'Fart',
 15: 'Oboe',
 16: 'Flute',
 17: 'Cough',
 18: 'Telephone',
 19: 'Bark',
 20: 'Chime',
 21: 'Bass_drum',
 22: 'Bus',
 23: 'Squeak',
 24: 'Scissors',
 25: 'Harmonica',
 26: 'Gong',
 27: 'Microwave_oven',
 28: 'Burping_or_eructation',
 29: 'Double_bass',
 30: 'Shatter',
 31: 'Fireworks',
 32: 'Tambourine',
 33: 'Cowbell',
 34: 'Electric_piano',
 35: 'Meow',
 36: 'Drawer_open_or_close',
 37: 'Applause',
 38: 'Acoustic_guitar',
 39: 'Violin_or_fiddle',
 40: 'Finger_snapping'}

In [5]:
dicts_ch={}
for i, v in dicts.items():
    dicts_ch[v]=i
dicts_ch

{0: 'Hi-hat',
 1: 'Saxophone',
 2: 'Trumpet',
 3: 'Glockenspiel',
 4: 'Cello',
 5: 'Knock',
 6: 'Gunshot_or_gunfire',
 7: 'Clarinet',
 8: 'Computer_keyboard',
 9: 'Keys_jangling',
 10: 'Snare_drum',
 11: 'Writing',
 12: 'Laughter',
 13: 'Tearing',
 14: 'Fart',
 15: 'Oboe',
 16: 'Flute',
 17: 'Cough',
 18: 'Telephone',
 19: 'Bark',
 20: 'Chime',
 21: 'Bass_drum',
 22: 'Bus',
 23: 'Squeak',
 24: 'Scissors',
 25: 'Harmonica',
 26: 'Gong',
 27: 'Microwave_oven',
 28: 'Burping_or_eructation',
 29: 'Double_bass',
 30: 'Shatter',
 31: 'Fireworks',
 32: 'Tambourine',
 33: 'Cowbell',
 34: 'Electric_piano',
 35: 'Meow',
 36: 'Drawer_open_or_close',
 37: 'Applause',
 38: 'Acoustic_guitar',
 39: 'Violin_or_fiddle',
 40: 'Finger_snapping'}

In [14]:
Y_train['onehot'] = Y_train['trans'].apply(lambda x: to_categorical(x,num_classes=len(labels)))
Y_train

,fname,label,manually_verified,trans,onehot
0,00044347.wav,Hi-hat,0,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,001ca53d.wav,Saxophone,1,1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,002d256b.wav,Trumpet,0,2,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,0033e230.wav,Glockenspiel,1,3,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,00353774.wav,Cello,1,4,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
5,003b91e8.wav,Cello,0,4,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
6,003da8e5.wav,Knock,1,5,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
7,0048fd00.wav,Gunshot_or_gunfire,1,6,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
8,004ad66f.wav,Clarinet,0,7,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
9,0063ab88.wav,Computer_keyboard,0,8,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."


In [19]:
train_Y = np.array(Y_train['onehot'].tolist())
train_Y.shape

(9473, 41)

In [6]:
xgb_mfcc = xgb.XGBClassifier(tree_method='gpu_hist',predictor='gpu_predictor') #n_jobs=11,
train_X = X_train.reshape(X_train.shape[0],-1)
# xgb_mfcc.fit(train_X,Y_train['trans'].values)

In [6]:
lgb_fbank = lgb.LGBMClassifier(n_jobs=11,boosting_type='gbdt',max_depth=-1,is_unbalance=True,num_class=41,metric='acc') #,device='gpu' n_jobs=11,
train_X2 = X_train2.reshape(X_train2.shape[0],-1) #n_jobs=-1,
# train_X = X_train.reshape(X_train.shape[0],-1)
lgb_fbank.fit(train_X2,Y_train['trans'].values)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        is_unbalance=True, learning_rate=0.1, max_depth=-1, metric='acc',
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=11, num_class=41, num_leaves=31,
        objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
        silent=True, subsample=1.0, subsample_for_bin=200000,
        subsample_freq=1)

In [8]:
vc = VotingClassifier(estimators=[('xgb',xgb_mfcc),('lgb',lgb_fbank)],n_jobs=11,voting='soft',flatten_transform=True)
vc.fit(train_X2,Y_train['trans'].values) #n_jobs=2

Process ForkPoolWorker-2:
Process ForkPoolWorker-3:
Process ForkPoolWorker-6:
Process ForkPoolWorker-7:
Process ForkPoolWorker-10:
Process ForkPoolWorker-4:
Process ForkPoolWorker-8:
Process ForkPoolWorker-9:
Process ForkPoolWorker-5:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


KeyboardInterrupt: 

In [7]:
test_Y = np.load('feature/fbank/X_test.npy')
# test_Y.shape
Y_test = test_Y.reshape(9400,-1)

In [8]:
ans = lgb_fbank.predict(Y_test)

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [9]:
np.save('result/lgb_naive.npy',ans)

In [11]:
pickle.dump(lgb_fbank,open('model/lgb_naive.pkl','wb'))

In [30]:
# kk = pickle.load(open('model/lgb_vc.pkl','rb'))
# kk.predict(Y_test)

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([30, 39, 21, ...,  4, 21,  0])

In [12]:
up = pd.read_csv('data/sample_submission.csv')
up

,fname,label
0,00063640.wav,Laughter Hi-Hat Flute
1,0013a1db.wav,Laughter Hi-Hat Flute
2,002bb878.wav,Laughter Hi-Hat Flute
3,002d392d.wav,Laughter Hi-Hat Flute
4,00326aa9.wav,Laughter Hi-Hat Flute
5,0038a046.wav,Laughter Hi-Hat Flute
6,003995fa.wav,Laughter Hi-Hat Flute
7,005ae625.wav,Laughter Hi-Hat Flute
8,007759c4.wav,Laughter Hi-Hat Flute
9,008afd93.wav,Laughter Hi-Hat Flute


In [13]:
up['ans1'] = pd.Series(ans)
up['trans']=up['ans1'].map(dicts_ch)
up

,fname,label,ans1,trans
0,00063640.wav,Laughter Hi-Hat Flute,32,Tambourine
1,0013a1db.wav,Laughter Hi-Hat Flute,16,Flute
2,002bb878.wav,Laughter Hi-Hat Flute,21,Bass_drum
3,002d392d.wav,Laughter Hi-Hat Flute,21,Bass_drum
4,00326aa9.wav,Laughter Hi-Hat Flute,15,Oboe
5,0038a046.wav,Laughter Hi-Hat Flute,5,Knock
6,003995fa.wav,Laughter Hi-Hat Flute,30,Shatter
7,005ae625.wav,Laughter Hi-Hat Flute,26,Gong
8,007759c4.wav,Laughter Hi-Hat Flute,18,Telephone
9,008afd93.wav,Laughter Hi-Hat Flute,1,Saxophone


In [ ]:
# def modify(string):

In [14]:
# up.label.str.replace('Laughter',up.trans)
up['label'] = up['trans']
up[['fname','label']].to_csv("result/naive_fbank_lgb_submission.csv", index=False)